#  Dev Journal

I'm trying a new thing, using jupyter notebook as a development journal for this project. Suppose will see how it goes. Anyway, up until now I managed to understand wav files better and play them back. I also have some rudimentry GUI, with a waveform display.

## 23/3/20 - Session 4

I figured in this session I'll improve our playback capabilities. Aiming for concurrent playback and pausing (so it won't hang up program), GUI buttons for controlling and playing just the selected section.

So play now invokes a thread for play_sample. The separation of code to gui.py and sampler.py is in the way of implementing a pause. I can bring the implementation higher so the code resides in the Player widget, but then it will have to change if I decide to use something other than pyaudio later. On the other hand, the playback is a loop, and how do I pause a loop from outside?

reading around. found all these crazy libraries for python and audio handling. remember, we want to build our own instead of using someone else's library. found a musicplayer library in python with nice separation between gui and player. I don't want to use it as it will require too much learning, but perhaps I can take that idea. So now I need to write a simple terminal music player?

Perhaps use pyglet, seems like there's a media player there. Let's give it a try here

In [15]:
def pause_all():
    from pyglet.media.codecs.base import Source
    for p in Source._players:
        p.pause()
pause_all()

Hacking around. First approach too create my own sources without saving to file and reloading is totally evil, accessing `source._data`. Next is trying to create the source myself. Better yet, you can feed `load(...)` your own decoder, so I could abuse it to create new sources. Should I use my own implementation of sample and decode it?

Wrote SampleSource and SampleDecoder. They are based on my current Sample implementation. Also, currently all that is in `sampler.py` so hopefully separated from GUI. Now I'm trying to figure out how to work with players a bit more.

Weird error when import one of the pyglet modules. Gonna call it a day. Overall been working for around 5-6 hrs. Not bad.

## 24/3/20 - Session 5
After some investigation, the problems on the surface happens here,

`pyglet\media\codecs\wmf.py:829 ole32.CoInitializeEx(None, COINIT_MULTITHREADED)`

This fails because somehow it was called before with a different parameter than COINIT_MULTITHREADED. This exception is ignored though, and since it's in __init__ the member self.MFShutdown is not set, and later the program crashes because of this missing attribute. Commenting out line 829 seems to avoid the problem, but definitely just a hack, so worth further investigation.

Anyhow, after some minor bug fixing the porgram works, but playback is buggy, just a small part is played, so this is our next stop.


In [14]:
import pyglet
import sampler
pyglet.app.run()
sample = sampler.open_file('Delvon Lamarr Organ Trio - Move On Up (Live on KEXP).wav')
source = pyglet.media.load('', file=sample, decoder=sampler.SampleDecoder())
source2 = pyglet.media.load('Delvon Lamarr Organ Trio - Move On Up (Live on KEXP).wav')

p = source2.play()

p.play()
# p.pause()

KeyboardInterrupt: 

Okay it might actually be a bug in pyglet or something? Cause the original code I wrote above also has this strange looping. So it seems we need to tick pyglet's clock, ie advance it's event loop.

Huh, apparently there's a Qt media player... Well, if it doesn't workout with pyglet we'll have something else to try.

Found out about QTimer, a way to add your own code to Qt event loop. I can now play, pause and stop, with current timestamp displayed. Pretty sweet. Still some nagging issues, program hangs when running not in debug. Also waveform is super slow, and just found out the timestamp keeps running when song is over.

Gonna call it a day. Next session I'll integrate the waveform display.